In [ ]:
#Bibliothek pandas zur Datenverarbeitung importieren
import pandas as pd

#Bibliothek numpy für Berechnungen importieren
import numpy as np
from numpy import tanh

#Bibliotheken plotly und cufflinks zur Datenvisualisierung importieren
from plotly.offline import iplot
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()

#Ein quadratsiches Layout für plotly-Graphiken wird definiert 
square_layout_scatter = go.Layout(xaxis={'title':'Breite','range':[-0.01,1.01]},
                   yaxis={'title':'Länge','range':[-0.01,1.01]},
                   height=810,
                   width=810)
square_layout_heatmap = go.Layout(xaxis={'title':'Breite','range':[-0.01,1.01]},
                   yaxis={'title':'Länge','range':[-0.01,100.01]},
                   height=810, width=810)

# Anwendungsbeispiel

Wir schauen uns wieder unser bekanntes Anwendungbeispiel an, aber wollen in diesem Notebook unser ausgereiftes Lernprinzip ausprobieren.

In [ ]:
#Importiere Daten
df_käfer = pd.read_csv('Käfer.csv', sep=';')


In [ ]:
#Viusalisiere Daten
df_käfer.iplot('scatter', mode='markers', 
               x='Breite', xTitle ='Breite', 
               y='Länge', yTitle = 'Länge', 
               categories= 'Insekt',
               layout = square_layout_scatter)

Wir erzeugen einen Datensatz <code>df_käfer_num</code>, in dem nicht die Käferbezeichnungen, sondern die numerischen targets angegeben werden.

In [ ]:
#Erzeuge numerische Daten
df_käfer_num = df_käfer.replace(['Raupe','Marienkäfer'],[-1,1])

df_käfer_num.head()

# Initialisieren des Neuronalen Netzwerks

Wir setzen wieder in gewohnter Form ein KNN auf, allerdings nutzen wir diesmal etwas willkürlichere Anfangskantengewichte. Wir achten lediglich darauf, dass die Kantengewichte relativ nah an der 0 liegen. Das ist vorteilhaft für den Lernprozess, da wir nicht wissen welche Gewichte am Ende einen positiven und welche einen negativen Wert bekommen. 

In [ ]:
#Festlegung der Kantengewichte
w1 = 0.1

w2 = 0.1

w3 = -0.1


In [ ]:
#Funktion für den Output des KNN 
def knn_output(x1, x2): 
    z = tanh(w1*x1+w2*x2)
    y = tanh(w3*z)
    return y

In [ ]:
#Bibliothek zur Visualisierung neuronaler Netzwerke
from Manual.lib.PrintNetwork import print_network

#Gewichte in Matrixdarstellung bringen 
gewichte_matrix = [pd.DataFrame(data={'in_1': [w1], 'in_2': [w2]}, index=['h_1']),
                   pd.DataFrame(data={'h_1': [w3]}, index=['out'])]

#Visualisiere das Neuronale Netzwerks
print_network([2, 1, 1], gewichte_matrix, ['out'], 2)

Wenn man die folgende Heatmap mit unserem Datensatz vergleicht ist zu sehen ist, dass das KNN mit den gewählten Kantengewichten nicht sehr gut funktioniert.

In [ ]:
#Heatmap erstellen
heatmap = pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]
heatmap.index = [y/100 for y in range(100)]

#Heatmap visualisieren
heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale = 'RdBu', 
              layout = square_layout_heatmap, zmax = 0.05, zmin= -0.05)

Bevor wir das KNN lernen lassen, schreiben wir noch eine Funktion, um den Anteil der korrekten Klassifikationen berechnen zu können. Dies wird wie gewohnt umgesetzt. Dabei wird ein Output größer gleich Null als Klasse 'Marienkäfer' interpretiert und ein Output kleiner Null als Klasse 'Raupe'. 

In [ ]:
#Funktion, die den Anteil der korrekten Klassifikationen berechnet
def accuracy():
    vorhersagen_liste = []

    for i in range(len(df_käfer)):

        output = knn_output(df_käfer.iloc[i]['Breite'],df_käfer.iloc[i]['Länge'])

        if output >= 0:
            vorhersagen_liste.append('Marienkäfer')
        elif output < 0:
            vorhersagen_liste.append('Raupe')

    df_käfer_test = df_käfer.copy()
    df_käfer_test['Vorhersagen'] = vorhersagen_liste

    print('Anteil richtiger Vorhersagen:', sum(df_käfer_test['Insekt'] == df_käfer_test['Vorhersagen'])/len(df_käfer_test))

accuracy()

Nur 50 % der Beispiele werden richtig klassifiziert. D. h. das KNN ist aktuell so gut wie ein Zufallsgenerator der das Ergebnis rät. 

# Definieren des Lernverfahrens

Nun wollen wir die alles entscheidende Funktion für das Lernen bzw. das anapassen der Gewichte definieren.

In den folgenden Zellen werden zunächst die Lernrate und dann die Funktion für das anpassen der Kantengewichte definiert. Dafür werden die an der Tafel hergeleiteten Aktualisierungsregeln genutzt.

In [ ]:
#Lernrate
lr = 0.03

In [ ]:
#Funktion für  das Update der Gewichte 
#Input x1: Breite; x2: Länge; t: target; w1, w2, w3 : Kantengewichte

def gewichte_update(x1, x2, t ,w1, w2, w3):
    
    #Berechnung der Neuronenaktivierung bis zum Output
    z = tanh(w1*x1+w2*x2)
    y = tanh(w3*z)
    
    #Aktualisierung der Gewichte
    w1 += lr*w3*(t-y)*(1.0-z**2)*x1    
    w2 += lr*w3*(t-y)*(1.0-z**2)*x2    
    w3 += lr*(t-y)*(1.0-y**2)*z        
    
    return w1, w2, w3

# Lernprozess 

## (1. Epoche)

In der folgenden Zelle werden alle Beispiele aus dem Datensatz einmal betrachtet und zur Aktualisierung der Gewichte genutzt. 

Dies nennt man eine **Epoche** des Lernprozesses des KNN.

In [ ]:
#Aktualisieren der Gewichte 
for j in range(len(df_käfer_num)):

    x1 = df_käfer_num.iloc[j][0]
    x2 = df_käfer_num.iloc[j][1]
    t = df_käfer_num.iloc[j][2]

    w1, w2, w3 = gewichte_update(x1, x2, t, w1, w2, w3)

Lass dir die aktualisierten Kantengewichte, die Visualisierung des Netzes und die Output-Heatmap in den folgenden drei Zellen anzeigen:

In [ ]:
#Anzeigen der Gewichte
display(w1, w2, w3)

In [ ]:
#Gewichte in Matrixdarstellung bringen 
gewichte_matrix = [pd.DataFrame(data={'in_1': [w1], 'in_2': [w2]}, index=['h_1']),
                   pd.DataFrame(data={'h_1': [w3]}, index=['out'])]

#Visualisiere das Neuronale Netzwerks
print_network([2, 1, 1], gewichte_matrix, ['out'], 2)

In [ ]:
#Heatmap erstellen
heatmap=pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]
heatmap.index = [y/100 for y in range(100)]

#Heatmap visualisieren
heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge',colorscale = 'RdBu', 
              layout = square_layout_heatmap, zmax = 0.1, zmin= -0.1)

In [ ]:
accuracy()

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Die erste Lernepoche wurde absolviert. Vergleiche den Anteil der richtigen Vorhersagen (Accuracy) vor und nach der Epoche. Ist das KNN deutlich besser geworden? 

Vergleiche die Output-Heatmap mit der Heatmap weiter oben (Ausgangszustand). Was fällt auf?
    </div> 
</div>




ANTWORT HIER EINFÜGEN

## (2. Epoche)

Eine Epoche reicht häufig nicht aus, da bei jeder Gewichtsaktualsierung nur eine sehr kleine Änderung vorgenommen wird. Somit werden die Daten erneut geutzt, um weitere Epoche im Lernprozess durchzuführen. 

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Führe die weiteren Epochen des Lernprozesses wie oben durch und beobachte jeweils die Veränderung der Kantengewichte, die Änderung der Output-Heatmap und die Änderung in der Vorhersagegenauigkeit (Accruracy).

Beschreibe nach jeder Epoche, welche Veränderungen festzustellen sind.
    </div> 
</div>





In [ ]:
#Aktualisieren der Gewichte 
for j in range(len(df_käfer_num)):

    x1=df_käfer_num.iloc[j][0]
    x2=df_käfer_num.iloc[j][1]
    t=df_käfer_num.iloc[j][2]

    w1, w2, w3 = gewichte_update(x1, x2, t, w1, w2, w3)

In [ ]:
#Anzeigen der Gewichte
display(w1, w2, w3)

In [ ]:
#Gewichte in Matrixdarstellung bringen 
gewichte_matrix = [pd.DataFrame(data={'in_1': [w1], 'in_2': [w2]}, index=['h_1']),
                   pd.DataFrame(data={'h_1': [w3]}, index=['out'])]

#Visualisiere das Neuronale Netzwerks
print_network([2, 1, 1], gewichte_matrix, ['out'], 2)

In [ ]:
#Heatmap erstellen
heatmap=pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]
heatmap.index = [y/100 for y in range(100)]

#Heatmap visualisieren
heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale = 'RdBu', 
              layout = square_layout_heatmap, zmax = 0.3, zmin= -0.3)

In [ ]:
accuracy()

BESCHREIBUNG DER VERÄNDERUNGEN HIER EINFÜGEN

## (3. Epoche)

In [ ]:
#Aktualisieren der Gewichte 
for j in range(len(df_käfer_num)):

    x1=df_käfer_num.iloc[j][0]
    x2=df_käfer_num.iloc[j][1]
    t=df_käfer_num.iloc[j][2]

    w1, w2, w3 = gewichte_update(x1, x2, t, w1, w2, w3)

In [ ]:
#Anzeigen der Gewichte
display(w1, w2, w3)

In [ ]:
#Gewichte in Matrixdarstellung bringen 
gewichte_matrix = [pd.DataFrame(data={'in_1': [w1], 'in_2': [w2]}, index=['h_1']),
                   pd.DataFrame(data={'h_1': [w3]}, index=['out'])]

#Visualisiere das Neuronale Netzwerks
print_network([2, 1, 1], gewichte_matrix, ['out'], 2)

In [ ]:
#Heatmap erstellen
heatmap=pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]
heatmap.index = [y/100 for y in range(100)]

#Heatmap visualisieren
heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale = 'RdBu', 
              layout = square_layout_heatmap, zmax = 1, zmin= -1)

In [ ]:
accuracy()

BESCHREIBUNG DER VERÄNDERUNGEN HIER EINFÜGEN

## (4. Epoche)

In [ ]:
#Aktualisieren der Gewichte 
for j in range(len(df_käfer_num)):

    x1=df_käfer_num.iloc[j][0]
    x2=df_käfer_num.iloc[j][1]
    t=df_käfer_num.iloc[j][2]

    w1, w2, w3 = gewichte_update(x1, x2, t, w1, w2, w3)

In [ ]:
#Anzeigen der Gewichte
display(w1, w2, w3)

In [ ]:
#Gewichte in Matrixdarstellung bringen 
gewichte_matrix = [pd.DataFrame(data={'in_1': [w1], 'in_2': [w2]}, index=['h_1']),
                   pd.DataFrame(data={'h_1': [w3]}, index=['out'])]

#Visualisiere das Neuronale Netzwerks
print_network([2, 1, 1], gewichte_matrix, ['out'], 2)

In [ ]:
#Heatmap erstellen
heatmap=pd.DataFrame()

for x in range(100):
    heatmap[str(x/100)] = [knn_output(y/100,x/100) for y in range(100)]
heatmap.index = [y/100 for y in range(100)]

#Heatmap visualisieren
heatmap.iplot('heatmap', xTitle='Breite', yTitle ='Länge', colorscale = 'RdBu', layout = square_layout_heatmap, zmax = 1, zmin= -1)

In [ ]:
accuracy()

BESCHREIBUNG DER VERÄNDERUNGEN HIER EINFÜGEN

<div class="panel panel-info">
    <div class="panel-heading">
        <b>Aufgabe</b>
    </div>
    <div class="panel-body">
Sollten noch weitere Epochen durchgeführt werden? Begründe.
    </div> 
</div>





ANTWORT HIER EINFÜGEN